In [1]:
import os
from os.path import expanduser
import socket
import sys


# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "awi-models")
elif running_system[0:1] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"awi-models")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
# IMPORT PY-FESOM MODULES
sys.path.append("codes/modules")
sys.path.append("codes/pyfesom-master/") # add pyfesom to search path
#########################################

computer =  blogin1
working directory set to /home/hbkoziel/awi-models


In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
#import colormaps as cmaps
from matplotlib.colors import ListedColormap
import pyfesom as pf

In [3]:
# PLOT CONFIGURATION #

# load colormaps #
cmap = plt.get_cmap('RdBu_r')
# figure export 
export_plot = True
# figure export definition
dpicnt=150
# choose simulation
runid='Arc12'
# choose date
first_year = 1980
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(0,11,12).astype(int)
# choose depth
get3d = False
# load the given biological tracer #
var_id1, var_id2 = 'diags_bio1_mean','diags_bio2_mean'
# export CSV
export_csv = True
########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'
savepath    = wd_path+'/figures/'+runid+'/'
meshpath    = wd_path+'/data/mesh/meshArc4.5/'
outputpath = '/scratch/usr/hbkoziel/Arc12/CSV/'

# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

directory /home/hbkoziel/awi-models/figures/Arc12/ already existing


In [4]:
mesh = fesom_mesh(meshpath, get3d = get3d)

Load 2d part of the grid in 3 second(s)


In [5]:
CHL  = np.zeros((len(years),len(mesh.x2)))

for ye in range(0,len(years)):
    print years[ye]
    ncfile  = resultpath+runid+'.'+str(years[ye])+'.bio.mean.nc'
    f       = Dataset(ncfile, 'r')    
    if years[ye] < 2000:
        chl1     = f.variables[var_id1][:,0:mesh.n2d]
        #chl2     = f.variables[var_id2][:,0:mesh.n2d]
        chl=(chl1)*12.01
        time = np.arange(0,360,30)
        data_int = np.trapz(chl,time,axis=0)
    elif years[ye] >= 2000 and years[ye] < 2015:
        chl1     = f.variables[var_id1][:,0:mesh.n2d]
        #chl2     = f.variables[var_id2][:,0:mesh.n2d]
        chl=(chl1)*12.01
        time = np.arange(0,350,2)
        data_int = np.trapz(chl,time,axis=0)
    else:
        chl1     = f.variables[var_id1][:,0:mesh.n2d]
        #chl2     = f.variables[var_id2][:,0:mesh.n2d]
        chl=(chl1)*12.01
        time = np.arange(0,334,2)
        data_int = np.trapz(chl,time,axis=0)
        
    CHL[ye,:]=data_int
    
data2=CHL.mean(axis=0)
print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
print 'Mean of surface: ',np.mean(data2[0:len(mesh.x2)])
print 'Max and min: ',np.max(data2),np.min(data2)
    
    
result_fit = np.ma.polyfit(years,CHL,1)
trend = result_fit[0,:]
    
if export_csv == True:
    np.savetxt(outputpath+'NPPannualphy_'+str(first_year)+'_'+str(last_year)+'_timeseries.csv', CHL, delimiter=";")
    np.savetxt(outputpath+'NPPannualphy_'+str(first_year)+'_'+str(last_year)+'_mean.csv', data2, delimiter=";")
    np.savetxt(outputpath+'NPPannualphy_'+str(first_year)+'_'+str(last_year)+'_trend.csv', trend, delimiter=";")
    print 'exporting done ...'

1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  14827.4673612
Max and min:  135462.935202 -17.415270307
exporting done ...
